In [2]:
import model
import dataset

import albumentations as A
from albumentations.augmentations.transforms import Flip

import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [4]:
if __name__ == '__main__':
    trainer = Trainer(max_epochs = 6, gpus = 1, precision=16, amp_level='O1',deterministic=True)
    
    train_tr = A.Compose([
        A.CenterCrop(200,200,always_apply=True),
        Flip()
    ])
    
    val_tr = A.Compose([
        A.CenterCrop(200,200,always_apply=True)
    ])
    
    model = model.Classifier({'lr':3e-4,'batch_size':64,'train_tr':train_tr,'val_tr':val_tr})
    
    trainer.fit(model)
    trainer.test(model)
    out = trainer.predict(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


Loaded pretrained weights for efficientnet-b3



  | Name   | Type         | Params
----------------------------------------
0 | resnet | EfficientNet | 10.7 M
----------------------------------------
10.7 M    Trainable params
0         Non-trainable params
10.7 M    Total params
42.797    Total estimated model params size (MB)
Restored states from the checkpoint file at lightning_logs/version_21/checkpoints/epoch=4-step=3124.ckpt


Validation sanity check: 0it [00:00, ?it/s]

D:\Rover\ch1\model.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = F.softmax(x)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1': 0.9994653463363647, 'test_loss': 0.0011666949139907956}
--------------------------------------------------------------------------------


Predicting: 625it [00:00, ?it/s]

In [5]:
import pandas as pd
def writeSub(p):
    labelmap = {0:'perseverance',1:'curiosity'}
    test_df = pd.read_csv('D:/rover/ch1/data/sample_submission.csv')
    output_list = p.int().tolist()
    output_list = [labelmap[i] for i in output_list]
    test_df['label'] = output_list
    test_df.to_csv('submission.csv',index = False)

In [6]:
output = torch.tensor([])

for i in range(len(out)):
    output = torch.cat((output,torch.tensor(out[i][1]).argmax(1)))

In [7]:
writeSub(output)